In [11]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Read in our nonprofit data
charity_df = pd.read_csv("charity_data.csv")

In [2]:
charity_df

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,996012607,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,996015768,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [7]:
# Evaluation for data preprocessing
# Model target: IS_SUCCESSFUL
# Features: APPLICATION_TYPE, AFFLIATION, USE_CASE, ORGANIZATION, INCOME_AMT, SPECIAL_CONSIDERATIONS, ASK_AMT 
# To Remove: EIN, NAME, CLASSIFICATION, STATUS

# Drop columns for unnecessary features
charity_cleaned_df = charity_df.drop(['EIN', 'NAME', 'CLASSIFICATION', 'STATUS', 'APPLICATION_TYPE'], axis=1)

In [8]:
# Generate our categorical variable list
charity_cat = charity_cleaned_df.dtypes[charity_cleaned_df.dtypes == "object"].index.tolist()

In [9]:
# Check the number of unique values in each column
charity_cleaned_df[charity_cat].nunique()

AFFILIATION               6
USE_CASE                  5
ORGANIZATION              4
INCOME_AMT                9
SPECIAL_CONSIDERATIONS    2
dtype: int64

In [12]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(charity_cleaned_df[charity_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(charity_cat)
encode_df.head()

,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
# Merge one-hot encoded features and drop the originals
charity_cleaned_df = charity_cleaned_df.merge(encode_df,left_index=True, right_index=True)
charity_cleaned_df = charity_cleaned_df.drop(charity_cat,1)
charity_cleaned_df.head()

,ASK_AMT,IS_SUCCESSFUL,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,USE_CASE_CommunityServ,USE_CASE_Heathcare,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,5000,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,5000,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,6692,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [19]:
# Split our preprocessed data into our features and target arrays
y = charity_cleaned_df["IS_SUCCESSFUL"].values
X = charity_cleaned_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [20]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [21]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 8)                 224       
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 6         
Total params: 275
Trainable params: 275
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 110261.0703 - accuracy: 0.5231
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 20411.3730 - accuracy: 0.4883
Epoch 3/100
804/804 [==============================] - 1s 975us/step - loss: 3772.0081 - accuracy: 0.5093
Epoch 4/100
804/804 [==============================] - 1s 955us/step - loss: 17260.6406 - accuracy: 0.5055
Epoch 5/100
804/804 [==============================] - 1s 996us/step - loss: 13740.1162 - accuracy: 0.5145
Epoch 6/100
804/804 [==============================] - 1s 992us/step - loss: 18032.6348 - accuracy: 0.4920
Epoch 7/100
804/804 [==============================] - 1s 979us/step - loss: 4967.1929 - accuracy: 0.5073
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 22325.6484 - accuracy: 0.5238
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 4641.8975 - accuracy: 0.5329
Epoch 10/100
804/804 [=========================

804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 78/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 79/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 80/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 81/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6910 - accuracy: 0.5321
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 85/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 86/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


268/268 - 0s - loss: 0.6909 - accuracy: 0.5332
Loss: 0.690945029258728, Accuracy: 0.5331778526306152


In [25]:
# Adjusting for one hidden layer to improve accuracy
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 8)                 224       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 9         
Total params: 233
Trainable params: 233
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [27]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 6957.9209 - accuracy: 0.5072
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 6078.0371 - accuracy: 0.5229
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 4821.6997 - accuracy: 0.5130
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 6882.2480 - accuracy: 0.5260
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 6271.7339 - accuracy: 0.5050
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 7598.8159 - accuracy: 0.5200
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 1031.2899 - accuracy: 0.5720
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 6607.8018 - accuracy: 0.5058
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 2557.4019 - accuracy: 0.5201A: 0s - loss: 2192.3921 - accurac
Epoch 10/100
804/804 [=========

804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 78/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321: 0s - loss: 0.6911 - accuracy: 0.53
Epoch 79/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 80/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 81/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 85/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 86/100
804/804 [==============================] - 1s 

In [28]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6910 - accuracy: 0.5332
Loss: 0.690953254699707, Accuracy: 0.5331778526306152


In [29]:
# Adjusting-  changing activation in hidden layer
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 8)                 224       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 9         
Total params: 233
Trainable params: 233
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [31]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 939us/step - loss: 0.6920 - accuracy: 0.5278
Epoch 2/100
804/804 [==============================] - 1s 974us/step - loss: 0.6913 - accuracy: 0.5321
Epoch 3/100
804/804 [==============================] - 1s 950us/step - loss: 0.6913 - accuracy: 0.5321
Epoch 4/100
804/804 [==============================] - 1s 924us/step - loss: 0.6913 - accuracy: 0.5321
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6913 - accuracy: 0.5321
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5321: 0s - loss: 0.6918 - accuracy: 
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6914 - accuracy: 0.5321
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6913 - accuracy: 0.5321
Epoch 10/100
804/804 [=============================

804/804 [==============================] - 1s 1ms/step - loss: 0.6914 - accuracy: 0.5321
Epoch 81/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6914 - accuracy: 0.5321
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6914 - accuracy: 0.5321
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6913 - accuracy: 0.5321
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6913 - accuracy: 0.5321
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6914 - accuracy: 0.5321
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6913 - accuracy: 0.5321
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 89/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6913 - accuracy: 

In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6910 - accuracy: 0.5332
Loss: 0.6910260915756226, Accuracy: 0.5331778526306152


In [39]:
# Adjusting- adding hidden layer and additional nodes, changing to relu activation
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  12
hidden_nodes_layer2 = 8

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 12)                336       
_________________________________________________________________
dense_17 (Dense)             (None, 8)                 104       
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 9         
Total params: 449
Trainable params: 449
Non-trainable params: 0
_________________________________________________________________


In [40]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [41]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 41710.7891 - accuracy: 0.5260
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 6438.2832 - accuracy: 0.5046
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 1392.8781 - accuracy: 0.4848
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 2440.3469 - accuracy: 0.4933
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 2851.1270 - accuracy: 0.4939
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 1005.5229 - accuracy: 0.5099
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 1791.7783 - accuracy: 0.5170
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 854.0837 - accuracy: 0.5027
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 1749.1633 - accuracy: 0.5040
Epoch 10/100
804/804 [==============================] - 1s 2ms/s

804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 80/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 81/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 

In [42]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6909 - accuracy: 0.5332
Loss: 0.690945565700531, Accuracy: 0.5331778526306152


In [47]:
# Adjusting- adding a hidden layer
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  12
hidden_nodes_layer2 = 8
hidden_nodes_layer3 = 4

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 12)                336       
_________________________________________________________________
dense_23 (Dense)             (None, 8)                 104       
_________________________________________________________________
dense_24 (Dense)             (None, 4)                 36        
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 5         
Total params: 481
Trainable params: 481
Non-trainable params: 0
_________________________________________________________________


In [48]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [49]:
# Train the model with 50 epochs
fit_model = nn.fit(X_train,y_train,epochs=50)

Epoch 1/50
804/804 [==============================] - 1s 1ms/step - loss: 42214.2578 - accuracy: 0.5213
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.7273 - accuracy: 0.4679
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.6996 - accuracy: 0.4679
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.6922 - accuracy: 0.5233
Epoch 5/50
804/804 [==============================] - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 9/50
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 10/50
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.

In [50]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6910 - accuracy: 0.5332
Loss: 0.6909524202346802, Accuracy: 0.5331778526306152
